In [1]:
import folium
import geopandas as gpd
from IPython.display import display, Image
import requests
import warnings
warnings.filterwarnings('ignore')

# ATL08 v003 Entwine Point Tiles (EPT)

This notebook demonstrates using the ATL08 IceSAT-2 [Entwine Point Tile (EPT)](https://entwine.io/entwine-point-tile.html) store.

### What are Entwine Point Tiles?

Entwine Point Tiles are a cloud-optimized octree data format for storing and visualizing massive point clouds efficiently. This format is gaining a lot of momentum and interest with an active development community.


### Visualizing with potree

Potree is a tool for visualizing EPT stores directly.

[potree.entwine.io: ATL08](https://potree.entwine.io/data/view.html?r=%22https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/ATL08_ARD-beta___001/global/ept%22)

In [4]:
img_src = "potree-global.gif"
Image(url = img_src)

### Visualizing with the 3D Tiles Service (OGC)

Cesium is a 3D tool which can be used to visualize point clouds alongside 2D data using a 3D Tiles Service.

* [3D Tiles API Endpoint](https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)
* [Demo using Cesium](http://cesium.entwine.io/?url=https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)

In [3]:
img_src = "cesium-global.gif"
Image(url = img_src)

## Querying ATL08 EPT using the OGC Features API

We can use the features service for some basic querying of the EPT Store.

### Query By Bounding Box

In [14]:
# Format a request to the API
api_url = "https://obnrh8ozt0.execute-api.us-east-2.amazonaws.com/collections/Global/items"

# bbox should be defined as xmin, xmax, (min value z), ymin, ymax, (max value z)
# Make a request for a bounding box over Peru
bbox="-77,-26,300,-73,0,500"

payload = {
    "f": "json",
    "limit": 100,
    "bbox": bbox,
}

r = requests.get(api_url, params = payload)

In [15]:
# Get the results directly into a Geo Data Frame (saving to file not required but recommended)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

,geometry,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,...,GpsTime,Red,Green,Blue,ScanChannel,ClassFlags,ElevationLow,HeightAboveGround,OffsetTime,OriginId
0,POINT Z (-75.58000 -8.80000 310.06000),-75.58,-8.80,310.06,0.0,1.0,1.0,0.0,0.0,0.0,...,1.240313e+09,10.0,95.0,101.0,0.0,0.0,289.127,42.708,41512918.0,29028.0
1,POINT Z (-74.17000 -11.57000 471.59000),-74.17,-11.57,471.59,0.0,1.0,1.0,0.0,0.0,0.0,...,1.245357e+09,27.0,107.0,111.0,0.0,0.0,488.210,25.316,46556779.0,37945.0
2,POINT Z (-73.48000 -10.88000 329.13000),-73.48,-10.88,329.13,0.0,1.0,1.0,0.0,0.0,0.0,...,1.258205e+09,25.0,106.0,110.0,0.0,0.0,314.425,35.354,59404688.0,58028.0
3,POINT Z (-74.16000 -10.18000 304.93000),-74.16,-10.18,304.93,0.0,1.0,1.0,0.0,0.0,0.0,...,1.258550e+09,12.0,96.0,101.0,0.0,0.0,302.224,20.358,59749777.0,58629.0
4,POINT Z (-73.51000 -9.49000 319.50000),-73.51,-9.49,319.50,0.0,1.0,1.0,0.0,0.0,0.0,...,1.266051e+09,17.0,100.0,105.0,0.0,0.0,296.021,27.648,67251450.0,71742.0


In [16]:
m = folium.Map(
    location=[adf.centroid[0].y, adf.centroid[0].x],
    zoom_start=10,
    tiles='Stamen Terrain'
)


folium.GeoJson(
    adf,
    name = "geojson"
).add_to(m)

m

### Query by Granule Id

In [17]:
granule_id = 'ATL08_20181014035224_02370107_003_01'
payload = {
    "f": "json",
    "origin": granule_id,
}

r = requests.get(api_url, params = payload)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

,geometry,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,...,GpsTime,Red,Green,Blue,ScanChannel,ClassFlags,ElevationLow,HeightAboveGround,OffsetTime,OriginId
0,POINT Z (-4.84000 11.91000 420.14000),-4.84,11.91,420.14,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,212.0,180.0,0.0,0.0,422.889,9.414,24724580.0,24.0
1,POINT Z (-5.01000 10.87000 492.45000),-5.01,10.87,492.45,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,163.0,125.0,0.0,0.0,489.439,9.171,24724596.0,24.0
2,POINT Z (-5.19000 8.80000 375.03000),-5.19,8.80,375.03,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,234.0,206.0,0.0,0.0,369.275,13.547,24724629.0,24.0
3,POINT Z (-5.54000 5.31000 64.02000),-5.54,5.31,64.02,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,17.0,100.0,105.0,0.0,0.0,59.659,8.826,24724683.0,24.0
4,POINT Z (-5.53000 5.13000 43.44000),-5.53,5.13,43.44,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,5.0,91.0,98.0,0.0,0.0,30.929,14.915,24724686.0,24.0


## More query options with pdal

The [documentation](https://maap-project.readthedocs.io/en/latest/query/testing-ept-stores.html#PDAL-Pipelines) provides additional options for how to query with PDAL (Point Cloud Data Abstraction Library).

# Thank You!

**Big thanks to** Aaron Kaulfus, Alex Mandel, Chuck Daniels, David Bitner, Hai, Kaylin Bugbee, Slesa, Sam Ayers and Seth Vincent with whom none of this would be possible.